The goal of this notebook is to play with the models exposed from Allen NLP library listed here: https://github.com/allenai/allennlp/tree/v0.3.0

Steps:

1. Install docker for mac - https://docs.docker.com/docker-for-mac/install/
2. Download and run the container, making sure to map port 8000 to port 8000 to expose it outside of docker
 * ```docker run -it --rm -p 8000:8000 allennlp/allennlp```
3. Within the image, run the webserver on port 8000
 * `allennlp/run serve --port 8000`
4.  Now in your browser, hit http:/localhost:8000 for the webserver

See https://github.com/allenai/allennlp/tree/master/allennlp/service for basic documentation.

## Running Saved Docker Image

You will want to run the web server once, and while it's running, save the container to a new image so that the models are stored within it, and don't need downloading each time. To do this, with the container still running, get the container id using `docker ps`. Then run:

`docker commit <containerid> allennlp_test/1.0`

To commit the container's state to a new image called allennelp_test/1.0. 
Then kill the current container, and run the new image:

`docker run --it --rm -p 8000:8000 allennlp_test/1.0`

# API Documentation (from link)

`GET /models` returns a list of the available models.

`POST /predict/<model_name>` asks the specified model for a prediction, based on the data in the request body. Each model expects different inputs:

* Semantic Role Labeling: `{"sentence": "..."}`
* Bidaf (question answering): `{"paragraph": "...", "question": "..."}`
* Snli: `{"premise": "...", "hypothesis": "..."}`

In [42]:
import requests
import json

url = "http://localhost:8000"
requests.get(url + "/models").json()

{'models': ['machine-comprehension',
  'semantic-role-labeling',
  'textual-entailment',
  'coreference-resolution',
  'named-entity-recognition']}

In [48]:
def call_api(method, payload):
    headers = {'content-type': "application/json"}
    url = "http://localhost:8000"
    resp = requests.request("POST", 
                            url + "/predict/{method}".format(method=method), 
                            data=json.dumps(payload), 
                            headers=headers)    
    return resp.json()

## Textual Entailment API Call

In [47]:
payload = {"premise":"Changes in the amount of CO2 threaten the delicate balance required to keep corals healthy. So if coral don't get the right amount, they could turn white",
           "hypothesis":"Increases in carbon dioxide cause coral bleaching"}

pprint(call_api("textual-entailment", payload))

{'label_logits': [2.001786470413208, -2.3041346073150635, -0.35969048738479614],
 'label_probs': [0.9027150273323059, 0.012176231481134892, 0.08510876446962357]}


## Machine Comprehension API Call

In [53]:
payload = {"passage":"The world’s temperatures are constantly changing. This is one reason coral bleaching goes on the in the world",
           "question":"What causes coral bleaching"}

pprint(call_api("machine-comprehension", payload))

{'best_span': [0, 6],
 'best_span_str': 'The world’s temperatures are constantly changing',
 'span_end_logits': [-8.260040283203125,
                     -7.483365535736084,
                     -6.194005012512207,
                     0.31115949153900146,
                     -7.251424312591553,
                     -6.025007724761963,
                     0.7867436408996582,
                     -1.9502825736999512,
                     -7.713595867156982,
                     -11.265436172485352,
                     -9.199939727783203,
                     -8.536502838134766,
                     -9.122642517089844,
                     -7.163054943084717,
                     -9.061155319213867,
                     -12.157005310058594,
                     -10.687776565551758,
                     -12.400985717773438,
                     -11.765970230102539,
                     -8.368474960327148],
 'span_end_probs': [6.968861271161586e-05,
                    0.000151519023347